In [1]:
import sys
import os
sys.path.append(os.path.abspath('/root/project')) # add root path to sys.path
sys.path.append(os.path.abspath('/usr/local/lib/python3.10/dist-packages'))
from util import util_constants
import database_init as database_init
import f4_evaluate.evaluate_scene as evaluate_scene



/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read all test spatial files with json.load(file) as list
PATH_CURR_FOLDER = os.path.abspath('') 
PARH_DATA_ANNOTATION = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_annotation'))
TEST_SPATIAL_ANNOTATION_PATH_MANUAL = PARH_DATA_ANNOTATION + '/spatial_testset/manual'
TEST_SPATIAL_ANNOTATION_PATH_SEMI = PARH_DATA_ANNOTATION + '/spatial_testset/semi'    

print(TEST_SPATIAL_ANNOTATION_PATH_MANUAL)
print(TEST_SPATIAL_ANNOTATION_PATH_SEMI)
test_spatial_video_list = database_init.make_spatial_json_video_list(TEST_SPATIAL_ANNOTATION_PATH_MANUAL, TEST_SPATIAL_ANNOTATION_PATH_SEMI)
len(test_spatial_video_list)

/root/project/data_annotation/spatial_testset/manual
/root/project/data_annotation/spatial_testset/semi


71

In [3]:
#Sort test_spatial_video_list first
sorted_test_video_list = sorted(test_spatial_video_list, key=lambda x: x["video_id"])
dict_list = sorted_test_video_list
#iterate each dictionary. Do pairwise similarity comparison of 'spatial_data' 

# PAIRWISE FOR SCENE1 - id=0
data_0 = dict_list[0]['spatial_data']
names0 = names1 = [obj["object_name"] for obj in data_0]
for i in range(1, len(dict_list)):  
    data_i = dict_list[i]['spatial_data']  
    print(f"{i}: {dict_list[i]['video_id']}")
    evaluate_scene.compare_scene_graphs_dict(data_0, data_i)    

# # PAIRWISE COMPARISON FOR ALL DOUBLE COMBINATION
# for i in range(len(dict_list)):    
#     data_i = dict_list[i]['spatial_data']
#     for j in range(len(dict_list)):
#         print(f"{j}: {dict_list[j]['video_id']}")
#         data_j = dict_list[j]['spatial_data']
#         evaluate_scene.compare_scene_graphs_dict(data_i, data_j)

#         #result_j = my_function(data_i, data_j)        
#         # if j == i:
#         #     print("same")
#         # else:
#         #     print(f"{j}th comparison")
#         #     # just do comparision
#         #     data_j = dict_list[j]['spatial_data']
#         #     evaluate_scene.compare_scene_graphs_dict(data_i, data_j)
#         #     #result_j = my_function(data_i, data_j)

1: 026dac2d-2ab3-4f9c-9e1d-6198db4fb080
🔍 Entity-Level Similarity:
  Precision: 0.11
  Recall:    0.14
  F1 Score:  0.12
  Matches:   [('table', 'table', 1.0000001192092896)]

🔗 Relationship-Level Similarity:
  Precision: 0.00
  Recall:    0.00
  F1 Score:  0.00
  Matches:   []
2: 02a06bf1-51b8-4902-b088-573e29fcd7ec
🔍 Entity-Level Similarity:
  Precision: 0.00
  Recall:    0.00
  F1 Score:  0.00
  Matches:   []

🔗 Relationship-Level Similarity:
  Precision: 0.00
  Recall:    0.00
  F1 Score:  0.00
  Matches:   []
3: 14bcb17c-f70a-41d5-b10d-294388084dfc
🔍 Entity-Level Similarity:
  Precision: 0.11
  Recall:    0.10
  F1 Score:  0.11
  Matches:   [('table', 'table1', 0.8066108226776123)]

🔗 Relationship-Level Similarity:
  Precision: 0.00
  Recall:    0.00
  F1 Score:  0.00
  Matches:   []
4: 1a894d3c-b3ef-448a-a3de-2b38677cef36
🔍 Entity-Level Similarity:
  Precision: 0.00
  Recall:    0.00
  F1 Score:  0.00
  Matches:   []

🔗 Relationship-Level Similarity:
  Precision: 0.00
  Recall:  